In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, SpatialDropout1D, SimpleRNN
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, RNN
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

Using TensorFlow backend.


In [2]:
test = pd.read_csv('updated_merged_data.csv')

In [3]:
test.drop(columns=['Unnamed: 0'], inplace=True)
test.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_30,pc_31,pc_32,pc_33,pc_34,pc_35,pc_36,pc_37,pc_38,signal_y
0,171,105,1,0.366667,assuming acceleration of to but in a comfortab...,-0.031750,-0.028850,-0.030290,-0.018736,-0.060675,...,0.017885,0.045822,-0.006862,-0.012954,-0.005468,-0.100572,-0.029374,0.041363,0.035932,stay
1,77,113,1,0.200000,is capable of transporting satellite to orbit ...,-0.059331,-0.099826,-0.154772,-0.040355,0.072285,...,-0.011667,0.003882,0.018002,-0.019813,-0.035809,-0.034079,-0.014687,-0.008986,-0.019845,stay
2,171,6,0,0.000000,yup,-0.002330,0.033983,0.069531,-0.014177,-0.037774,...,0.015722,-0.025726,0.065735,0.041686,-0.021751,-0.014138,-0.037047,-0.050031,-0.084722,stay
3,171,7,0,0.000000,part,-0.005050,0.024117,0.043362,-0.001804,-0.016279,...,-0.009052,-0.008601,0.001285,-0.007619,-0.005289,-0.025700,-0.008989,-0.011967,-0.004665,stay
4,171,96,1,0.650000,fly to most place on earth in under min and an...,-0.031167,-0.027725,-0.012331,-0.039909,-0.070462,...,0.020042,0.030526,-0.039462,0.088186,-0.012277,0.033849,0.040576,0.051975,-0.031236,stay


In [4]:
test.tweet[:5]

0    assuming acceleration of to but in a comfortab...
1    is capable of transporting satellite to orbit ...
2                                                  yup
3                                                 part
4    fly to most place on earth in under min and an...
Name: tweet, dtype: object

In [5]:
test.tweet = test.tweet.astype(str)

In [140]:
max_fatures = 44
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['tweet'].values)
X = tokenizer.texts_to_sequences(test['tweet'].values)
X = pad_sequences(X)

In [141]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 128)           5632      
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 15, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_238 (Dense)            (None, 3)                 591       
Total params: 261,023
Trainable params: 261,023
Non-trainable params: 0
_________________________________________________________________
None


In [142]:
y = pd.get_dummies(test['signal_y']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2564, 15) (2564, 3)
(641, 15) (641, 3)


In [143]:
model.fit(X_train, y_train, epochs=200, validation_split=0.1, batch_size=32, verbose=1)

Train on 2307 samples, validate on 257 samples
Epoch 1/200
2307/2307 [==============================] - 5s 2ms/step - loss: 1.0888 - acc: 0.4053 - val_loss: 1.0902 - val_acc: 0.4047
Epoch 2/200
2307/2307 [==============================] - 2s 875us/step - loss: 1.0823 - acc: 0.4148 - val_loss: 1.0919 - val_acc: 0.3969
Epoch 3/200
2307/2307 [==============================] - 2s 896us/step - loss: 1.0743 - acc: 0.4278 - val_loss: 1.1062 - val_acc: 0.3813
Epoch 4/200
2307/2307 [==============================] - 2s 898us/step - loss: 1.0695 - acc: 0.4300 - val_loss: 1.1072 - val_acc: 0.3619
Epoch 5/200
2307/2307 [==============================] - 2s 899us/step - loss: 1.0656 - acc: 0.4348 - val_loss: 1.1099 - val_acc: 0.3619
Epoch 6/200
2307/2307 [==============================] - 2s 900us/step - loss: 1.0647 - acc: 0.4408 - val_loss: 1.1092 - val_acc: 0.3658
Epoch 7/200
2307/2307 [==============================] - 2s 942us/step - loss: 1.0600 - acc: 0.4361 - val_loss: 1.1158 - val_acc: 0.3

2307/2307 [==============================] - 2s 976us/step - loss: 0.2991 - acc: 0.8505 - val_loss: 2.6284 - val_acc: 0.3541
Epoch 120/200
2307/2307 [==============================] - 2s 1ms/step - loss: 0.2994 - acc: 0.8526 - val_loss: 2.5973 - val_acc: 0.3424
Epoch 121/200
2307/2307 [==============================] - 3s 1ms/step - loss: 0.2904 - acc: 0.8539 - val_loss: 2.5609 - val_acc: 0.3502
Epoch 122/200
2307/2307 [==============================] - 2s 874us/step - loss: 0.2899 - acc: 0.8587 - val_loss: 2.6451 - val_acc: 0.3619
Epoch 123/200
2307/2307 [==============================] - 2s 873us/step - loss: 0.2911 - acc: 0.8535 - val_loss: 2.6948 - val_acc: 0.3658
Epoch 124/200
2307/2307 [==============================] - 2s 860us/step - loss: 0.2951 - acc: 0.8544 - val_loss: 2.6761 - val_acc: 0.3580
Epoch 125/200
2307/2307 [==============================] - 2s 1ms/step - loss: 0.2953 - acc: 0.8531 - val_loss: 2.6794 - val_acc: 0.3619
Epoch 126/200
2307/2307 [======================

In [10]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.16
acc: 0.34


---

In [15]:
test.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_30,pc_31,pc_32,pc_33,pc_34,pc_35,pc_36,pc_37,pc_38,signal_y
0,171,105,1,0.366667,assuming acceleration of to but in a comfortab...,-0.031750,-0.028850,-0.030290,-0.018736,-0.060675,...,0.017885,0.045822,-0.006862,-0.012954,-0.005468,-0.100572,-0.029374,0.041363,0.035932,stay
1,77,113,1,0.200000,is capable of transporting satellite to orbit ...,-0.059331,-0.099826,-0.154772,-0.040355,0.072285,...,-0.011667,0.003882,0.018002,-0.019813,-0.035809,-0.034079,-0.014687,-0.008986,-0.019845,stay
2,171,6,0,0.000000,yup,-0.002330,0.033983,0.069531,-0.014177,-0.037774,...,0.015722,-0.025726,0.065735,0.041686,-0.021751,-0.014138,-0.037047,-0.050031,-0.084722,stay
3,171,7,0,0.000000,part,-0.005050,0.024117,0.043362,-0.001804,-0.016279,...,-0.009052,-0.008601,0.001285,-0.007619,-0.005289,-0.025700,-0.008989,-0.011967,-0.004665,stay
4,171,96,1,0.650000,fly to most place on earth in under min and an...,-0.031167,-0.027725,-0.012331,-0.039909,-0.070462,...,0.020042,0.030526,-0.039462,0.088186,-0.012277,0.033849,0.040576,0.051975,-0.031236,stay


In [17]:
XX = test.drop(columns=['signal_y', 'tweet'])

In [19]:
# XX.head()

In [21]:
# y = pd.get_dummies(test['signal_y']).values
X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size = 0.20, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2564, 42) (2564, 3)
(641, 42) (641, 3)


In [31]:
model2 = Sequential()
model2.add(Dense(20, input_dim=42, kernel_initializer='normal', activation='tanh'))
model2.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model2.add(Dropout(0.01))
model2.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model2.add(Dropout(0.01))
model2.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model2.add(Dropout(0.01))
model2.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model2.add(Dense(10, activation='tanh'))
model2.add(Dense(10, activation='tanh'))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 20)                860       
_________________________________________________________________
dense_47 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_48 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_49 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_50 (Dense)             (None, 3)                 33        
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
model2.fit(X_train, y_train, epochs=120, validation_split=.1, batch_size=20, verbose=1)

Train on 2307 samples, validate on 257 samples
Epoch 1/120
2307/2307 [==============================] - 1s 369us/step - loss: 1.0900 - acc: 0.4027 - val_loss: 1.0929 - val_acc: 0.3930
Epoch 2/120
2307/2307 [==============================] - 0s 70us/step - loss: 1.0871 - acc: 0.4083 - val_loss: 1.0885 - val_acc: 0.4047
Epoch 3/120
2307/2307 [==============================] - 0s 78us/step - loss: 1.0855 - acc: 0.4101 - val_loss: 1.0864 - val_acc: 0.4047
Epoch 4/120
2307/2307 [==============================] - 0s 95us/step - loss: 1.0856 - acc: 0.4105 - val_loss: 1.0883 - val_acc: 0.4047
Epoch 5/120
2307/2307 [==============================] - 0s 80us/step - loss: 1.0845 - acc: 0.4105 - val_loss: 1.0889 - val_acc: 0.4047
Epoch 6/120
2307/2307 [==============================] - 0s 80us/step - loss: 1.0849 - acc: 0.4109 - val_loss: 1.0887 - val_acc: 0.4008
Epoch 7/120
2307/2307 [==============================] - 0s 78us/step - loss: 1.0844 - acc: 0.4105 - val_loss: 1.0894 - val_acc: 0.4008


Epoch 120/120
2307/2307 [==============================] - 0s 65us/step - loss: 1.0618 - acc: 0.4283 - val_loss: 1.0996 - val_acc: 0.4008


In [136]:
classifier = Sequential()
classifier.add(Dense(42, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(21, activation = 'relu'))
classifier.add(Dense(21, activation = 'relu'))
classifier.add(Dense(10, kernel_regularizer=regularizers.l2(0.05), activation='relu'))
# classifier.add(Dense(10, kernel_regularizer=regularizers.l2(0.05), activation='relu'))
# classifier.add(Dense(5, kernel_regularizer=regularizers.l2(0.05), activation='relu'))
# classifier.add(Dense(5, activation = 'tanh'))
classifier.add(Dense(2, activation = 'softmax'))
adam = optimizers.Adam(lr=.01, decay=0.0005)

classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy']) 

In [137]:
classifier.fit(X_train.values, y_train, epochs=50, validation_split=.1, batch_size=20, verbose=1)

Train on 2307 samples, validate on 257 samples
Epoch 1/50
2307/2307 [==============================] - 3s 1ms/step - loss: 1.2233 - acc: 0.3975 - val_loss: 1.1033 - val_acc: 0.4047
Epoch 2/50
2307/2307 [==============================] - 0s 134us/step - loss: 1.0918 - acc: 0.4114 - val_loss: 1.0889 - val_acc: 0.4047
Epoch 3/50
2307/2307 [==============================] - 0s 134us/step - loss: 1.0857 - acc: 0.4114 - val_loss: 1.0857 - val_acc: 0.4047
Epoch 4/50
2307/2307 [==============================] - 0s 140us/step - loss: 1.0848 - acc: 0.4114 - val_loss: 1.0851 - val_acc: 0.4047
Epoch 5/50
2307/2307 [==============================] - 0s 132us/step - loss: 1.0849 - acc: 0.4114 - val_loss: 1.0849 - val_acc: 0.4047
Epoch 6/50
2307/2307 [==============================] - 0s 135us/step - loss: 1.0849 - acc: 0.4114 - val_loss: 1.0849 - val_acc: 0.4047
Epoch 7/50
2307/2307 [==============================] - 0s 135us/step - loss: 1.0845 - acc: 0.4114 - val_loss: 1.0859 - val_acc: 0.4047
Epo

In [138]:
classifier.evaluate(X_test.values, y_test, batch_size=20)

641/641 [==============================] - 0s 138us/step


[1.0890361503952193, 0.40249609910009804]

In [82]:
# from keras import losses

In [139]:
# losses??

In [ ]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1)